In [ ]:
import pandas as pd

In [ ]:
cafe_data = pd.read_csv('/content/drive/MyDrive/미니 프로젝트/data/카페_train.csv')

In [ ]:
df = cafe_data.dropna(axis=1)
df.head()

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트,상담번호,상담내순번
0,1,s,고객님 혹시 무슨 문제 있으신가요?,카페,5,q,m,제품_불량_질문,9252,9
1,2,c,네 이상해요,카페,5,a,m,제품_불량_질문,9252,10
2,3,c,빙수 위에 뿌려진 과일에서 쉰 냄새가 나는데 어떻게 하면 되죠?,카페,20591,q,n,제품_불량_질문,20591,1
3,4,s,"죄송합니다, 폐기처분 하고 새 빙수를 제조해드리겠습니다.",카페,20591,a,m,제품_불량_질문,20591,2
4,5,c,마카롱 주문했는데 마카롱이 엄청 딱딱해요.,카페,66996,q,n,제품_불량_질문,66996,1


In [ ]:
# 직원이 질문하는 경우 & 소비자가 대답하는 경우 삭제

x1 = df[(df['발화자']=='s')&(df['QA여부']=='q')].index
x2 = df[(df['발화자']=='c')&(df['QA여부']=='a')].index

df = df.drop(x1)
df = df.drop(x2)
df.reset_index(inplace = True, drop = True)

In [ ]:
# QA 정렬대로 뽑아오기
Q = []
A = []
label = []

for i in df.index:
  if df['QA여부'][i] == 'q' and df['QA여부'][i+1] == 'a':
    Q.append(df.loc[i]['발화문'])
    A.append(df.loc[i+1]['발화문'])
    label.append(df.loc[i]['인텐트'])

data= pd.DataFrame({'Q':Q,
                'A':A,
                'label' :label})


data

In [ ]:
# 중복 질문 제거
# 데이터: 72559 --> 중복 제거 후 데이터: 66041 
data.drop_duplicates(['Q'], inplace = True)
data.reset_index(inplace = True, drop = True)

In [ ]:
# -------------------------------------------
# label 삭제
# 우리 카페에 필요한 데이터만 남기기
# -------------------------------------------

# 매장, 멤버십, 예약, 웹사이트, AS, 제품_추천, 제품_용도, 비교, 재고, 교환, 제품_가격 삭제
del_label = data[data['label'].str.contains('매장|멤버십|예약|웹사이트|AS|제품_추천|제품_용도|비교|재고|교환|제품_가격')].index
data = data.drop(del_label)

data.reset_index(inplace = True, drop = True)

In [ ]:
# 필요 없는 데이터 제거
extra_q = data[data['Q'].str.contains('설빙|베스킨라빈스|스타벅스|나무|단청|도마|그림|파인트|쿼터|패밀리|하프갤런|텀블러')].index
data = data.drop(extra_q)

extra_a = data[data['A'].str.contains('설빙|베스킨라빈스|스타벅스|나무|단청|도마|그림|파인트|쿼터|패밀리|하프갤런|텀블러')].index
data = data.drop(extra_a)

data.reset_index(inplace=True, drop=True)

In [ ]:
# 날짜 묻는 질문/날짜로 대답하는 답변 제거
product_date_q = data[data['Q'].str.contains('1월|2월|3월|4월|5월|6월|7월|8월|9월|10월')].index
data = data.drop(product_date_q)

product_date_a = data[data['A'].str.contains('1월|2월|3월|4월|5월|6월|7월|8월|9월|10월')].index
data = data.drop(product_date_a)

data.reset_index(inplace=True, drop=True)

In [ ]:
# 배달/배송 관련 정보 제거
delivery_q = data[data['Q'].str.contains('배달|택배')].index
data = data.drop(delivery_q)

delivery_a = data[data['A'].str.contains('배달|택배')].index
data = data.drop(delivery_a)

delivery = data[data['label'].str.contains('배송')].index
data = data.drop(delivery)

data.reset_index(inplace=True, drop=True)

In [ ]:
# 수거 제거
waste_q = data[data['Q'].str.contains('수거')].index
data = data.drop(waste_q)

waste_a = data[data['A'].str.contains('수거')].index
data = data.drop(waste_a)

data.reset_index(inplace=True, drop=True)

In [ ]:
# 총 얼마예요? 제거
total_q = data[data['Q'].str.contains('총 얼마예요?')].index
data = data.drop(total_q)

data.reset_index(inplace=True, drop=True)

In [ ]:
# 품절 제거
absence_q = data[data['Q'].str.contains('품절')].index
data = data.drop(absence_q)

absence_a = data[data['A'].str.contains('품절')].index
data = data.drop(absence_a)

data.reset_index(inplace=True, drop=True)

In [ ]:
# 원두 삭제
coffee_q = data[data['Q'].str.contains('원두')].index
data = data.drop(coffee_q)

coffee_a = data[data['A'].str.contains('원두')].index
data = data.drop(coffee_a)

data.reset_index(inplace=True, drop=True)

In [ ]:
# 재료 변경 불가 안내 문구 학습
data.loc[data['Q'].str.contains('빼|교환|바꾸|추가|바꿔|뺄'), 'A'] = '현재 재료 변경은 불가능합니다.'

# 디카페인 불가 안내 문구 학습
data.loc[data['Q'].str.contains('디카페인|디 카페인'), 'A'] = '현재 디카페인 음료는 제공되지 않습니다.'

# 제품 불량 문의 안내 문구 학습
data.loc[data['label'].str.contains('제품_불량'), 'A'] = '죄송합니다. 카운터에서 확인 도와드리겠습니다.'

# 분할 결제 불가 안내 문구 학습
data.loc[data['Q'].str.contains('분할 결제|분할결제|나눠서 결제'), 'A'] = '현재 분할 결제가 불가능합니다. 다른 결제 수단을 선택해주세요.'

# 할인 불가 안내 문구 학습
data.loc[data['Q'].str.contains('할인|깎|깍'), 'A'] = '현재 할인은 불가합니다.'

# 칼로리 정보 불가능 안내 문구 학습
data.loc[data['Q'].str.contains('열량|칼로리'), 'A'] = '현재 칼로리 정보는 제공되지 않습니다. 카운터에 문의 바랍니다.'

# 행사 안내 문구 학습
data.loc[data['Q'].str.contains('행사|이벤트'), 'A'] = '현재 진행중인 행사가 없습니다. 다음 행사를 기다려주세요~^^'

In [ ]:
# Transformer 모델 사용시 label 제거
# data = data.drop(['label'], axis=1)

In [ ]:
### KoGPT 모델 사용시 label 필요

lst = list(data['label'])
lst2 = [i.split('_')[0] if i.split('_')[0] != '제품' else i.split('_')[0] +'_'+ i.split('_')[2] for i in lst]

data['label'] = lst2

In [ ]:
data_frame = data

In [ ]:
#################################
### label 항목 레이블, 원핫인코딩(의도분석에 이용)
#################################

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#레이블 인코딩
lab_encoder = LabelEncoder()
lab_encoder.fit(data_frame['label'].unique())
lst = lab_encoder.classes_
label_list = dict(zip(lst, range(len(lst))))

print(label_list)    #원래값-숫자 목록

{'결제': 0, '제품_요청': 1, '제품_질문': 2, '제품_확인': 3, '주문': 4, '포장': 5, '행사': 6}


In [ ]:
#레이블 데이터 적용
lab_in=lab_encoder.transform(data_frame['label'])   # label 값
ori_in=lab_encoder.inverse_transform(lab_in)         # 원래 값
data_frame['label'] = lab_in

In [ ]:
DATA_PATH = '/content/drive/MyDrive/미니 프로젝트/data/'
data.to_csv(DATA_PATH + 'cafe_qa_train.csv', index=False)